In [1]:
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
!pip install feedparser
!pip install newspaper3k

--2023-03-02 15:29:50--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/lesechos.json
Résolution de people.irisa.fr (people.irisa.fr)… 131.254.254.107
Connexion à people.irisa.fr (people.irisa.fr)|131.254.254.107|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 8276509 (7,9M) [application/json]
Enregistre : «lesechos.json.1»

lesechos.json.1     100%[===================>]   7,89M  6,20MB/s    ds 1,3s    

2023-03-02 15:29:52 (6,20 MB/s) - «lesechos.json.1» enregistré [8276509/8276509]

--2023-03-02 15:29:53--  https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/sources.json
Résolution de people.irisa.fr (people.irisa.fr)… 131.254.254.107
Connexion à people.irisa.fr (people.irisa.fr)|131.254.254.107|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 347 [application/json]
Enregistre : «sources.json.1»

sources.json.1      100%[===================>]     347  --.-KB/s    ds 0s      

2023-03-02 15:29:

In [2]:
!python3 -m spacy download fr_core_news_md

✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_md')


In [2]:
import spacy
nlp = spacy.load('fr_core_news_md')

## Question 1

In [4]:
from urllib.parse import urlparse

import json
f = open('lesechos.json')
data_lesechos = json.load(f)

# (a) se premunir contre le bloquage de commande
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# (b) recuperer le fichier RSS
import feedparser as fp
url = "https://www.lemonde.fr/rss/une.xml"
data = fp.parse(url)

for item in data.entries:
    print(item.title, item.published, item.link)

f = open('sources.json')
liste_rss_flux = json.load(f)
liste_rss_flux = list(liste_rss_flux.values())


liste_new_articles = []
# In lesechos.json, links are the keys but we cannot compare these links to the link we get when we parse RSS links.
# For example, a link from lesechos.json is 
# https://investir.lesechos.fr/marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee-1995443.php?xtor=RSS-24
# While a link from an item entry is 
# https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607
# So we need to compare only /marches/actualites/analyse-technique-toute-poursuite-de-la-consolidation-du-contrat-cac-40-sera-limitee
liste_url_article_already_in_base = [urlparse(temp).path[:-12] for temp in list(data_lesechos.keys())]
for url in liste_rss_flux:
    data_rss = fp.parse(url)
    for item in data_rss.entries:
        link = item.link
        res = urlparse(link)
        if res.path[:-8] not in liste_url_article_already_in_base:
            liste_new_articles.append(link)
    

Guerre en Ukraine en direct : pour la première fois depuis le début de l’invasion de l’Ukraine, des échanges de tirs ont été signalés sur le sol russe Thu, 02 Mar 2023 14:13:47 +0100 https://www.lemonde.fr/international/live/2023/03/02/guerre-en-ukraine-en-direct-la-russie-denonce-une-attaque-terroriste-de-saboteurs-ukrainiens-dans-la-region-de-briansk-kiev-evoque-une-provocation-deliberee-de-moscou_6163832_3210.html
Les débats sur la réforme des retraites démarrent au Sénat, Attal déclare « qu’ici il n’y a pas de ZAD, il n’y a que la République » Thu, 02 Mar 2023 16:23:49 +0100 https://www.lemonde.fr/politique/article/2023/03/02/reforme-des-retraites-les-debats-demarrent-au-senat-dans-une-atmosphere-plus-sereine_6163902_823448.html
Les dessous du voyage de Marine Le Pen au Sénégal Thu, 02 Mar 2023 06:02:24 +0100 https://www.lemonde.fr/politique/article/2023/03/02/les-dessous-du-voyage-de-marine-le-pen-au-senegal_6163837_823448.html
Service national universel : la possible généralisati

## Question 2

**Pour l’un des articles de votre flux RSS, regardez le code HTML de la page web. Pouvez-vous
identifier les zones contenant le titre de l’article ? le texte de l’article ?**

En analysant le code HTML d'un article sur `lesechos.fr`, on remarque que les balises pour le titre et le texte d'un article  ne sont pas clairement identifiables. On a par exemple des balises pour les titres au format suivant : `h1 class="sc-14kwckt-6 sc-14omazk-0 jtuxZx fsUfny"`.

**Que faudrait-il faire pour récupérer le texte de l’article ? En quoi le recours à la librairie newspaper est-il utile ?**

Utiliser ```newspaper``` nous simplifie la tâche : on peut accéder facilement aux élements de la page qui nous intéressent. De plus ce package est optimisé pour les articles de presse alors qu'un package comme BeautifulSoup serait plus contraignant ici.

In [5]:
import newspaper as np

url_one_article = liste_new_articles[0]

article = np.Article('https://investir.lesechos.fr/actu-des-valeurs/la-vie-des-actions/la-normalisation-des-ventes-de-velo-apres-le-boom-du-covid-laisse-des-traces-dans-les-comptes-de-thule-1905607')
article.download()
article.parse()

print(article.title, article.authors)
print(article.text)
print(article.top_image)

La normalisation des ventes de vélo, après le boom du Covid, laisse des traces dans les comptes de Thule []
Par Marjorie Encelot

Publié le 10 févr. 2023 à 15:31 Mis à jour le 10 févr. 2023 à 15:57

L’année 2021, pendant la crise sanitaire, a été « exceptionnelle » pour le vélo. Trop, finalement, quand à l’instar du suédois Thule Group (-17% à la Bourse de Stockholm, plus forte baisse aujourd’hui de l’indice européen Stoxx 600), on est une entreprise cotée en Bourse, qui vend des porte-vélos et des remorques pour vélo, et que la base de comparaison est historiquement élevée au moment de présenter ses comptes 2021.

Le quatrième trimestre, au cours duquel les ventes ont chuté de près de 21% à taux de change constants (+28% par rapport à 2019), a été particulièrement « challengeant », a reconnu le patron Jan Magnus Welander lors de la conférence téléphonique avec les analystes financiers, tant les trois derniers mois de 2021 avaient été « forts », aux Etats-Unis et au Canada essentiellem

## Question 3

**Reprendre votre programme de scrapping et le compléter de manière à mettre à jour votre
base de données avec le texte des articles du flux RSS et le lien vers l’image illustrant l’article
lorsqu’il y en a une. Vous pouvez inclure d’autres informations qui vous semblent utile pour un
traitement ultérieur.**

On traverse la liste des articles et on extrait les informations à l'aide de `newspaper` comme dans la question 2. Pour chaque article on ajoute les informations suivantes :
- le titre
- la date
- les auteurs
- le texte
- le lien de l'image principale

In [6]:
from datetime import datetime

for url in liste_new_articles:
    article = np.Article(url)
    article.download()
    article.parse()
    temp_dict = {'title' : article.title, 'date' : article.publish_date.strftime("%m/%d/%Y, %H:%M:%S"), 'author' : article.authors, 'category' : None, 'content' : article.text, 'image_link' : article.top_image, 'image_file' : None}
    data_lesechos[url] = temp_dict


In [7]:
with open("lesechos_updated.json", "w") as outfile:
    json.dump(data_lesechos, outfile)

## Question 4

Il est possible de télécharger des images depuis une url en python de la manière suivante :
> ```python
> import requests
> 
> img_data = requests.get(image_url).content
> with open('image_name.jpg', 'wb') as outfile:
>   outfile.write(img_data)
> ```

Pour préserver les images des articles, on pourrait les copier lorsque l'on réalise le scraping du texte du site. On sauvegarderait alors toutes les images dans un meme dossier `articles_img` en précisant le nom de l'article pour chaque image.

Pour simplifier le rangement et l'organisation des images, il serait possible de les ranger dans des dossiers par date ou par journal.

## Question 5

We can look into social medias such as Twitter. For example, we could follow the profile of the newspaper and for an article (in case they tweet about their article) look at some statistics such as views, RT, etc...

## Question 7

**À quoi correspondent les étiquettes IOB utilisées ? Expliquer brièvement une technique (de-
scription de la tâche, du modèle) pour la détection des entités nommées.**

Les étiquettes IOB (pour Inside–outside–beginning) correspondent au type d'entité nommée. 

Par exemple `emmanuel` sera étiquetté en `PER` et `paris` en `LOC`. 

## Question 8

**Écrire un programme qui traite l’ensemble des textes que vous aurez chargé depuis le fichier
lesechos.json et stock pour chaque article les entités trouvées et les informations afférentes
(type, span).**

In [8]:
import spacy
import json

data_lesechos_updated = json.load(open('lesechos_updated.json'))
list_text = [temp['content'] for temp in list(data_lesechos_updated.values())]
nlp = spacy.load('fr_core_news_md', disable=['tagging', 'parser'])
nlp.add_pipe('sentencizer')
docs = list(nlp.pipe(list_text))
list_entites_per_doc = []
for doc in docs:
    temp = []
    for s in doc.sents:
        temp_sentence = []
        for entity in s.ents:
            temp_sentence.append({'text' : entity, 'label' : entity.label_, 'start' : entity.start, 'end' : entity.end})
        temp.append(temp_sentence)
    list_entites_per_doc.append(temp)

In [9]:
list_entites_per_doc[0][1]

[{'text': Etats, 'label': 'LOC', 'start': 66, 'end': 67}]

## Question 9

**Pour chaque type d’entité, déterminer les 20 plus fréquentes dans la collection de documents.
Commentez le résultat obtenu. On mémorisera ces entités dans trois listes distinctes (une par
type d’entité).**

In [86]:
import itertools

flat_list = list(itertools.chain(*list_entites_per_doc))
flat_list = list(itertools.chain(*flat_list))

In [87]:
from collections import defaultdict

def takeFirst(elem):
    return elem[0]

label_counts = defaultdict(int)
for item in flat_list:
    label_counts[(str(item['text']), str(item['label']))] += 1

result = defaultdict(list)
for key, count in label_counts.items():
    result[key[1]].append((count, key[0]))

for label, items in result.items():
    items.sort(key = takeFirst ,reverse=True)
    result[label] = items[:20]

list_PER = result['PER']
list_ORG = result['ORG']
list_LOC = result['LOC']
list_MISC = result['MISC']


On compte pour chaque entité (i.e. le "texte") le nombre d'occurences et on trie en fonction de ce nombre.

## Question 10

In [89]:
def nb_co_occurence(pair_entities : tuple, types : tuple, same_sentence : bool = False):
    first_entity, second_entity = pair_entities
    first_type, second_type = types
    count = 0
    for doc in list_entites_per_doc:
        if same_sentence == False:
            indice_first = False
            indice_second = False
            doc = list(itertools.chain(*doc))
            for entity in doc:
                if str(entity['text']) == str(first_entity) and str(entity['label']) == str(first_type):
                    indice_first = True
                if str(entity['text']) == str(second_entity) and str(entity['label']) == str(second_type):
                    indice_second = True    
            if indice_first and indice_second:
                count += 1
        if same_sentence:
            for sentence in doc:
                indice_first = False
                indice_second = False
                for entity in sentence:
                    if str(entity['text']) == str(first_entity) and str(entity['label']) == str(first_type):
                        indice_first = True
                    if str(entity['text']) == str(second_entity) and str(entity['label']) == str(second_type):
                        indice_second = True    
                if indice_first and indice_second:
                    count += 1

            
    return count

In [90]:
nb_co_occurence(('Omicron', 'Cac 40'), ('PER', 'MISC'))

111

## Question 11

In [91]:
import itertools
from itertools import permutations
 
# initialize lists
list_per = [{'text' : e[1], 'label' : 'PER'} for e in list_PER]
list_org = [{'text' : e[1], 'label' : 'ORG'} for e in list_ORG]
list_loc = [{'text' : e[1], 'label' : 'LOC'} for e in list_LOC]
list_misc = [{'text' : e[1], 'label' : 'MISC'} for e in list_MISC]

big_list = list_per + list_org + list_loc + list_misc

combinations = []
for pair in itertools.combinations(big_list, 2):
    if pair[0]['label'] != pair[1]['label']:
        combinations.append([(pair[0]['text'], pair[1]['text']), (pair[0]['label'], pair[1]['label'])])

final = [list(pair) for pair in combinations]

list_occu = []
for l in final:
    first_entity, second_entity = l[0]
    first_type, second_type = l[1]
    list_occu.append((nb_co_occurence((first_entity, second_entity), (first_type, second_type)), l[0], l[1]))

list_occu = sorted(list_occu, reverse=True)

In [92]:
list_occu

[(575, ('Etats-Unis', 'Cac 40'), ('LOC', 'MISC')),
 (552, ('Fed', 'Cac 40'), ('ORG', 'MISC')),
 (533, ('Fed', 'Etats-Unis'), ('ORG', 'LOC')),
 (316, ('Etats-Unis', 'Dow Jones'), ('LOC', 'MISC')),
 (306, ('Jerome Powell', 'Fed'), ('PER', 'ORG')),
 (288, ('Etats-Unis', 'Bourse de Paris'), ('LOC', 'MISC')),
 (287, ('Fed', 'Dow Jones'), ('ORG', 'MISC')),
 (284, ('Fed', 'Bourse de Paris'), ('ORG', 'MISC')),
 (274, ('Fed', 'S&P 500'), ('ORG', 'MISC')),
 (273, ('Reuters', 'Etats-Unis'), ('ORG', 'LOC')),
 (265, ('New York', 'Cac 40'), ('LOC', 'MISC')),
 (260, ('Wall Street', 'Cac 40'), ('LOC', 'MISC')),
 (255, ('Etats-Unis', 'S&P 500'), ('LOC', 'MISC')),
 (249, ('Jerome Powell', 'Cac 40'), ('PER', 'MISC')),
 (249, ('Europe', 'Cac 40'), ('LOC', 'MISC')),
 (249, ('Chine', 'Cac 40'), ('LOC', 'MISC')),
 (243, ('Jerome Powell', 'Etats-Unis'), ('PER', 'LOC')),
 (242, ('Blandine Hénault', 'Reuters'), ('PER', 'ORG')),
 (229, ('Fed', 'New York'), ('ORG', 'LOC')),
 (224, ('Etats-Unis', 'Nasdaq Composite

## Question 12

In [93]:
import itertools
from itertools import permutations
 
# initialize lists
list_per = [{'text' : e[1], 'label' : 'PER'} for e in list_PER]
list_org = [{'text' : e[1], 'label' : 'ORG'} for e in list_ORG]
list_loc = [{'text' : e[1], 'label' : 'LOC'} for e in list_LOC]
list_misc = [{'text' : e[1], 'label' : 'MISC'} for e in list_MISC]

big_list = list_per + list_org + list_loc + list_misc

combinations = []
for pair in itertools.combinations(big_list, 2):
    if pair[0]['label'] != pair[1]['label']:
        combinations.append([(pair[0]['text'], pair[1]['text']), (pair[0]['label'], pair[1]['label'])])

final = [list(pair) for pair in combinations]

list_occu = []
for l in final:
    first_entity, second_entity = l[0]
    first_type, second_type = l[1]
    list_occu.append((nb_co_occurence((first_entity, second_entity), (first_type, second_type), same_sentence=True), l[0], l[1]))

list_occu = sorted(list_occu, reverse=True)

In [117]:
list_occu

[(180, ('Jerome Powell', 'Fed'), ('PER', 'ORG')),
 (142, ('Reuters', 'PARIS'), ('ORG', 'LOC')),
 (142, ('New York', 'Dow Jones'), ('LOC', 'MISC')),
 (138, ('New York', 'Nasdaq Composite'), ('LOC', 'MISC')),
 (106, ('Reuters', 'Etats-Unis'), ('ORG', 'LOC')),
 (105, ('Fed', 'Etats-Unis'), ('ORG', 'LOC')),
 (101, ('Hermès', 'LVMH'), ('PER', 'ORG')),
 (100, ('Reuters', 'Bourse de Tokyo'), ('ORG', 'MISC')),
 (94, ('Pivot', 'Support'), ('PER', 'ORG')),
 (87, ('New York', 'S&P 500'), ('LOC', 'MISC')),
 (83, ('Hermès', 'Kering'), ('PER', 'ORG')),
 (75, ('Christine Lagarde', 'BCE'), ('PER', 'ORG')),
 (68, ('Jerome Powell', 'Réserve fédérale américaine'), ('PER', 'ORG')),
 (66, ('Reuters', 'COVID-19'), ('ORG', 'MISC')),
 (55, ('Bourse', 'Chiffre'), ('LOC', 'MISC')),
 (53, ('Wall Street', 'Dow Jones'), ('LOC', 'MISC')),
 (52, ('Wall Street', 'Bourse de Paris'), ('LOC', 'MISC')),
 (45, ('Jackson Hole', 'Fed'), ('PER', 'ORG')),
 (45, ('Christine Lagarde', 'Banque centrale européenne'), ('PER', 'ORG

## Question 13

**Ecrivez une fonction qui prend en entrée deux entités et leurs types
respectif et qui analyse l’ensemble des documents où ces entités apparaissent dans la même
phrase de manière à établir la liste des verbes qui apparaissent entre ces deux entités dans ces
phrases.**

In [115]:
def verbs_between_entities(pair_entities : tuple, types : tuple):
    first_entity, second_entity = pair_entities
    first_type, second_type = types
    list_verbs = []
    for doc in list_entites_per_doc:
        for sentence in doc:
            indice_first = False
            indice_second = False
            for entity in sentence:
                if str(entity['text']) == str(first_entity) and str(entity['label']) == str(first_type):
                    indice_first = True
                    start_first = entity['start']
                    end_first = entity['end']
                if str(entity['text']) == str(second_entity) and str(entity['label']) == str(second_type):
                    indice_second = True
                    start_second = entity['start']
                    end_second = entity['end']    
            if indice_first and indice_second:
                start = min(start_first, start_second)
                end = max(end_first, end_second)
                true_sentence = sentence[0]['text'].sent
                portion = true_sentence[start:end]
                for token in portion:
                    if str(token.pos_) == 'VERB':
                        list_verbs.append(token)
    return list_verbs
                
    

# Question 14

**En utilisant la fonction de la question précédente, déterminez quelques couples d’entités et un
verbe associé qui vous paraissent faire sens.**

In [116]:
verbs_between_entities(('Wall Street', 'Cac 40'), ('LOC', 'MISC'))

[gagne,
 ancrés,
 connu,
 finit,
 dévissent,
 évolue,
 retombé,
 poursuivi,
 prononcée,
 poursuit,
 poursuit]

# Question 15

**En quoi les couples et verbes identifés à la question précédente peuvent-ils être utiles pour
trouver de nouvelles relations et/ou grouper des relations de même nature ?**

# Question 16